In [13]:
import numpy as np
import random

class WeightedQuickUnionUF(object):
    
    def __init__(self, n):
        self.store = []
        self.sz = []
        for i in range(n):
            self.store.append(i)
            self.sz.append(1)

    def find(self, p):
        if self.store[p] == p:
            return p
        return self.find(self.store[p])

    def union(self, p, q):
        p_root = self.find(p)
        q_root = self.find(q)
        if p_root == q_root:
            return
        if self.sz[p_root] < self.sz[q_root]:
            self.store[p_root] = q_root
            self.sz[q_root] += self.sz[p_root]
        else:
            self.store[q_root] = p_root
            self.sz[p_root] += self.sz[q_root]

    def is_connected(self, p, q):
        return self.find(p) == self.find(q)  

class percolation(object):
    
    def __init__(self, size):
        self.weighted_union =  WeightedQuickUnionUF(size*size + 2)
        self.site = [0] * (size*size+2)
        self.num = size
        self.open_num = 0
        for i in range(1, size + 1):
            self.weighted_union.union(0, i)
            self.weighted_union.union(size*size+1, size*size+1 - i)
        
    def open_site(self, row, col):
        if self.is_open(row,col):
            return
        self.open_num += 1
        cid = self.transfer_id(row,col)
        self.site[cid] = 1
        if row - 1 > 0 and self.is_open(row-1, col):
            id_up = self.transfer_id(row-1, col)
            self.weighted_union.union(cid, id_up)
        if row + 1 <= self.num and self.is_open(row+1, col):
            id_down = self.transfer_id(row+1, col)
            self.weighted_union.union(cid, id_down)
        if col - 1 > 0 and self.is_open(row, col-1):
            id_left = self.transfer_id(row, col-1)
            self.weighted_union.union(cid, id_left)
        if col + 1 <= self.num and self.is_open(row, col+1):
            id_right = self.transfer_id(row, col+1)
            self.weighted_union.union(cid, id_right)
        
        
    def is_open(self, row, col):
        index = self.transfer_id(row,col)
        return self.site[index] == 1
    
    def is_full(self,row,col):
        index = self.transfer_id(row,col)
        return self.site[index] == 1 and self.weighted_union.is_connected(0,index)
        
    def number_of_opensite(self):
        return self.open_num
    
    def is_percolate(self):
        return self.weighted_union.is_connected(0, self.num*self.num + 1)
    
    def transfer_id(self, row, col):
        if not (0 < row <= self.num):
            raise ValueError('the number is illegal for this project.')
            
        if not (0 < col <= self.num):
            raise ValueError('the number is illegal for this project.')
        return self.num * (row - 1) + col
        
    def random_col_row(self):
        while not self.is_percolate():
            row = random.randint(1, self.num)
            col = random.randint(1, self.num)
            self.open_site(row, col)
    
    
        
class percolationStats(object):
    
    def __init__(self,size, t):
        self.size= size
        self.number = t
        self.result = []
        for i in range(t):
            test = percolation(size)
            test.random_col_row()
            self.result.append(test.open_num/(size*size))
        self.np = np.array(self.result)
        
        
    def mean(self):
        return self.np.mean()
        
    def standdev(self):
        return self.np.var()
        
    def confidenceLo(self):
        return self.mean() - 1.96*self.standdev() / np.sqrt(self.number)
        
    def confidenceHi(self):
        return self.mean() + 1.96*self.standdev() / np.sqrt(self.number)

if __name__ == '__main__':
    n = 10
    t = 10
    percolation_stats = percolationStats(n, t)
    print('mean: {}'.format(percolation_stats.mean()))
    print('stddev: {}'.format(percolation_stats.standdev()))
    print('95% confidence interval: [{}, {}]'.format(
        percolation_stats.confidenceLo(), percolation_stats.confidenceHi()))

mean: 0.5670000000000001
stddev: 0.0030210000000000002
95% confidence interval: [0.5651275648009718, 0.5688724351990283]
